# 03: Polygon Triangulation

*Authors: Jan Erik Swiadek, Prof. Dr. Kevin Buchin*

This notebook serves as supplementary learning material for the course **Geometric Algorithms**.
It showcases implementations of algorithms and data structures presented in the lecture, and it elaborates on some practical considerations concerning their use.
Furthermore, it provides interactive visualisations and animations.

## Table of Contents

1. Introduction  
2. Setup  
3. Algorithms  
    3.1. Recursive Triangulation  
    3.2. Monotone Partitioning  
    3.3. Monotone Triangulation  
    3.4. Gallery Guarding  
4. References  

## 1. Introduction

We start by giving an overview of the essential definitions and facts:

* A **(simple) polygon** is a non-empty subset of $\mathbb{R}^2$ that's a union of two parts:
  The first part is a **boundary** in the form of a single closed path of line segments, which follows a sequence of points such that the segments have no intersections except for each pair of two consecutive segments sharing an endpoint.
  These characterising line segments and points are called the polygon's **edges** and **vertices**, respectively.
  The second part of the polygon is its **interior**, the open region enclosed by its boundary.

* A **triangulation** of a polygon is a decomposition into triangles with non-intersecting interiors, where all triangle vertices correspond to vertices of the original polygon.
  Of course, a triangle is a polygon with three vertices.

* According to the lecture, any simple polygon can be triangulated by adding line segments between polygon vertices such that the segments are contained in the polygon and don't intersect with each other or with the polygon edges anywhere except in their endpoints.
  These connecting line segments are called **diagonals**.

* A polygon is **$y$-monotone** if every horizontal line has a connected (or empty) intersection with the polygon.

See the following image for an example.

TODO: Add and describe image.

## 2. Setup

First let's do some setup. This is not very interesting, so you can skip to Section 3 if you want.

We now import everything we'll need throughout this notebook from external modules, including our module for generic data structures, our module for common geometric primitives and operations as well as our module for visualisation purposes. (The data structure and geometry modules will probably receive their own notebook later.)

In [ ]:
# Python standard library imports
from typing import Any, Optional, Callable, Iterator
from enum import Enum, auto
import copy

# External library imports
import numpy as np

# Data structure, geometry and visualisation module imports
from modules.data_structures import BinaryTreeDict, Comparator, ComparisonResult as CR
from modules.geometry import Point, PointSequence, DoublyConnectedSimplePolygon, Vertex, HalfEdge, Orientation as ORT
from modules.visualisation import (
    VisualisationTool, SimplePolygonInstance, LineSegmentsMode, MonotonePartitioningMode, ArtGalleryMode
)

Additionally, we create an object for our visualisation tool and register a few example instances.

In [ ]:
visualisation_tool = VisualisationTool(400, 400, SimplePolygonInstance())
canvas_size = min(visualisation_tool.width, visualisation_tool.height)

c = 0.5 * canvas_size
r = 0.75 * c

def rotate(point: Point, angle: float) -> Point:
    x = point.x * np.cos(angle) - point.y * np.sin(angle)
    y = point.x * np.sin(angle) + point.y * np.cos(angle)
    return Point(x, y)

clover_offsets = [
    Point(-0.4 * r, 0.4 * r), Point(-0.8 * r, 0.6 * r), Point(-r, 0.4 * r), Point(-r, 0.2 * r),
    Point(-0.9 * r, 0.0), Point(-r, -0.2 * r), Point(-r, -0.4 * r), Point(-0.8 * r, -0.6 * r)
]
clover_path = [Point(c, c) + rotate(offset, i * np.pi / 2) for i in range(0, 4) for offset in clover_offsets]
visualisation_tool.register_example_instance("Clover", DoublyConnectedSimplePolygon(clover_path))

s = c - r
t = c + r
u = (t - s) / 12

art_gallery_path = [
    Point(s, t), Point(s + 2 * u, t), Point(s + 2 * u, s + 9 * u), Point(s + 3 * u, s + 9 * u), Point(s + 3 * u, t),
    Point(s + 7 * u, t), Point(s + 7 * u, s + 7 * u), Point(s + 10 * u, s + 7 * u), Point(s + 10 * u, s + 8 * u),
    Point(s + 8 * u, s + 8 * u), Point(s + 8 * u, t), Point(t, t), Point(t, s + 4 * u), Point(s + 6 * u, s + 4 * u),
    Point(s + 6 * u, s + 3 * u), Point(t, s + 3 * u), Point(t, s), Point(s + 6 * u, s), Point(s + 6 * u, s + u),
    Point(s + 2 * u, s + u), Point(s + 2 * u, s), Point(s, s), Point(s, s + 3 * u), Point(s + 2 * u, s + 3 * u),
    Point(s + 2 * u, s + 4 * u), Point(s, s + 4 * u)
]
visualisation_tool.register_example_instance("Art Gallery", DoublyConnectedSimplePolygon(art_gallery_path))

def prongs_point(i: int) -> Point:
    return Point(s + ((i // 3) * 0.225 + (i % 3) * 0.05) * (t - s), t if i % 3 == 1 else s + 0.25 * (t - s))

prongs_path = [*(prongs_point(i) for i in range(1, 14)), Point(t, s), Point(s, s)]
visualisation_tool.register_example_instance("Prongs", DoublyConnectedSimplePolygon(prongs_path))

d = 0.2 * r

teeth_offsets = [Point(i * d, scale * r) for i, scale in zip(range(0, 6), (0.1, 1, 0.2, 1, 0.9, 1))]
shark_teeth_offsets = [*(Point(-offset.x, offset.y) for offset in reversed(teeth_offsets[1:])), *teeth_offsets]
shark_teeth_path = [Point(c, c) + rotate(offset, i * np.pi) for i in (0, 1) for offset in shark_teeth_offsets]
visualisation_tool.register_example_instance("Shark Teeth", DoublyConnectedSimplePolygon(shark_teeth_path))

## 3. Algorithms

The first algorithm in this notebook triangulates a polygon by recursively adding diagonals.
This is followed by an implementation of the more elaborate two-step triangulation procedure presented in the lecture.
Finally, the computed triangulations will be used for guarding art galleries.

### 3.1. Recursive Triangulation

In detail, the algorithm yielded by the proof of existence from the lecture (see also \[1, Theorem 3.1\]) works as follows:
It first searches for the leftmost vertex of the polygon.
Then it considers the triangle defined by this leftmost vertex and its two neighbouring vertices.
If there are no other polygon vertices inside the triangle or on the triangle edge between the neighbours, then that very edge is a valid diagonal and thus the neighbours get connected.
Otherwise, the algorithm connects the conflicting vertex farthest from said triangle edge to the leftmost vertex via a diagonal.
This process is repeated recursively on the subpolygons separated by added diagonals until all of them are triangles.

Checking whether a conflicting vertex exists and, if so, finding the desired one can be achieved by means of [area coordinates](https://en.wikipedia.org/wiki/Area_coordinates) with respect to the considered triangle.
The three area coordinate values of a point are easily computable using signed areas and they intuitively describe normalised distances to each of the triangle edges:
A coordinate value of $0$ indicates that the point lies on the line through the corresponding triangle edge, while a value of $1$ means that the point lies on the parallel line through the remaining triangle vertex.
Points inside the triangle or on its boundary are characterised by all coordinate values being contained in the closed interval $[0,1]$.

We've implemented an iterative variant of the algorithm that receives a simple polygon as a *doubly-connected edge list*, or *DCEL* for short.
This data structure was covered in the lecture on the line segment intersection problem and is, as its name implies, centered around edges.
Each edge is split into two *half-edges* pointing into opposite directions such that every half-edge is incident to exactly one face and is oriented with that face to its left.
This enables anticlockwise traversals of bounded faces, which correspond to the interiors of subpolygons in the scenario on hand.
Our implementation keeps track of each subpolygon that hasn't been handled yet through a representative half-edge incident to its interior.
The used DCEL admits adding a diagonal in asymptotically constant time, and the associated half-edges are then used as representatives.

Note that, unlike half-edges, vertices can be incident to the interiors of multiple subpolygons.
That's why our implementation operates on half-edges when adding a diagonal or searching for a vertex.
For instance, when searching for the leftmost vertex of a subpolygon, it rather searches for the unique half-edge that has the sought-after vertex as its origin and is incident to the subpolygon's interior — the leftmost edge of the subpolygon, so to speak.

In [ ]:
def recursive_triangulation(polygon: DoublyConnectedSimplePolygon) -> PointSequence:
    if polygon.has_diagonals():
        raise ValueError("Input polygon already has diagonals.")
    elif not polygon.is_simple():
        raise ValueError("Input polygon isn't simple.")

    diagonal_points = PointSequence()
    representative_edges = [polygon.topmost_vertex.edge]
    while representative_edges:
        leftmost_edge = get_leftmost_edge(representative_edges.pop())
        connection_edges = get_connection_edges(leftmost_edge)
        if connection_edges is not None:
            diagonal = polygon.add_diagonal(connection_edges[0], connection_edges[1])
            diagonal_points.append(diagonal.origin.point)
            diagonal_points.append(diagonal.destination.point)
            representative_edges.append(diagonal)
            representative_edges.append(diagonal.twin)

    return diagonal_points

def get_leftmost_edge(representative_edge: HalfEdge) -> HalfEdge:
    leftmost_edge, leftmost_vertex = representative_edge, representative_edge.origin
    edge = representative_edge.next
    while edge is not representative_edge:
        vertex = edge.origin
        if vertex.x < leftmost_vertex.x or (vertex.x == leftmost_vertex.x and vertex.y < leftmost_vertex.y):
            leftmost_edge, leftmost_vertex = edge, vertex
        edge = edge.next

    return leftmost_edge

def get_connection_edges(leftmost_edge: HalfEdge) -> Optional[tuple[HalfEdge, HalfEdge]]:
    edge = leftmost_edge.next.next
    if edge is leftmost_edge.prev:
        return None

    triangle = (leftmost_edge.prev.origin, leftmost_edge.origin, leftmost_edge.next.origin)
    conflicting_edge, max_coordinate = None, 0.0
    while edge is not leftmost_edge.prev:
        area_coordinates = calculate_area_coordinates(edge.origin, triangle)
        if all(0.0 <= coordinate <= 1.0 for coordinate in area_coordinates):
            if conflicting_edge is None or area_coordinates[1] > max_coordinate:
                conflicting_edge, max_coordinate = edge, area_coordinates[1]
        edge = edge.next

    if conflicting_edge is None:
        return leftmost_edge.prev, leftmost_edge.next
    return leftmost_edge, conflicting_edge

def calculate_area_coordinates(vertex: Vertex, triangle: tuple[Vertex, Vertex, Vertex]) -> tuple[float, float, float]:
    parallelogram_area = calculate_signed_area(triangle[0], triangle[1], triangle[2])
    return (
        calculate_signed_area(vertex, triangle[1], triangle[2]) / parallelogram_area,
        calculate_signed_area(triangle[0], vertex, triangle[2]) / parallelogram_area,
        calculate_signed_area(triangle[0], triangle[1], vertex) / parallelogram_area
    )

def calculate_signed_area(u: Vertex, v: Vertex, w: Vertex) -> float:
    p, q, r = u.point, v.point, w.point
    return (q - p).cross(r - p)

Traversing a polygon with $n$ vertices to search for the leftmost vertex and for potential conflicting vertices takes $O(n)$ operations.
Moreover, it can happen that no conflicting vertices are ever encountered and every step splits off a triangle.
Therefore, the worst-case running time complexity of Recursive Triangulation is in $O(n^2)$.

We now register the algorithm for visualisation.

In [ ]:
visualisation_tool.register_algorithm(
    "Rec. Triangulation", recursive_triangulation,
    LineSegmentsMode(vertex_radius = 0, highlight_radius = 0)
)

If you haven't used our interactive visualisation tool before, see the convex hull notebook for an explanation.
Note that generating random simple polygons is currently done using [2-opt](https://en.wikipedia.org/wiki/2-opt), which may take some time for polygons with many vertices.
Clicking on the canvas adds a vertex to the end of the existing polygon boundary path, except when doing so would lead to irreversible intersections in the path.
The provisional edge between the two vertices added first and last is displayed transparently.

TODO: Add static image.

In [ ]:
visualisation_tool.display()

***Takeaways:***

* A constructive proof of existence can result in an easily implementable algorithm.

* Area coordinates are a powerful tool for dealing with triangles.

### 3.2. Monotone Partitioning

Instead of splitting the input polygon into two arbitrary subpolygons and triangulating those recursively, a more careful approach is to partition it into subpolygons that admit a specialised triangulation algorithm.
One might first think of convex subpolygons, but finding such a partitioning is as hard as triangulating (see \[1, p. 49\]).
A suitable partitioning, as covered in the lecture and implemented in this section, yields $y$-monotone subpolygons.
The next section then presents how to triangulate those $y$-monotone subpolygons.

The partitioning algorithm employs the plane sweep technique, which we've already explained in the line segment intersection notebook.
This time, the event queue contains polygon vertices.
The status structure consists of all half-edges that are currently intersected by the sweep line and have the polygon interior to their right (see \[1, p. 52\]).
We again implement both as a balanced binary tree by writing respective comparator classes.
Because we want the sweep line to go from top to bottom like in the line segment intersection notebook, the event queue comparator is basically the same (see \[1, p. 50\]).
Though we don't use epsilon comparisons here, as this algorithm is quite robust.

In [ ]:
class EventQueueComparator(Comparator[Vertex]):
    def compare(self, item: Any, key: Vertex) -> CR:
        if not isinstance(item, Vertex):
            raise TypeError("Only vertices can be compared with event vertices.")
        elif item.point == key.point:
            return CR.MATCH
        elif item.y > key.y or (item.y == key.y and item.x < key.x):
            return CR.BEFORE
        else:
            return CR.AFTER

The status structure comparator is also similar to the one from the line segment intersection notebook, but less complicated since the edges of a simple polygon don't intersect or overlap.
In particular, two half-edges never change left-to-right order as long as they're in the status structure together.
That's why this comparator doesn't need a dynamic dependency on the current event vertex, and instead it's sufficient to compare an origin or destination vertex of one half-edge to the other half-edge if the vertex is contained in the latter half-edge's $y$-range.
Furthermore, the status structure will never contain two adjacent half-edges during the algorithm, while horizontal half-edges will always be deleted immediately after insertion.
Overall, we thus don't really need to consider any degenerate cases separately, making this comparator pretty straightforward as well.

In [ ]:
class StatusStructureComparator(Comparator[HalfEdge]):
    def compare(self, item: Any, key: HalfEdge) -> CR:
        if isinstance(item, Vertex):
            return self._compare_vertex_with_edge(item, key)
        elif isinstance(item, HalfEdge):
            return self._compare_edge_with_edge(item, key)
        else:
            raise TypeError("Only vertices and half-edges can be compared with status half-edges.")

    def _compare_vertex_with_edge(self, vertex: Vertex, edge: HalfEdge) -> CR:
        upper, lower = edge.upper_and_lower
        ort = vertex.point.orientation(lower.point, upper.point)
        if ort is ORT.LEFT:
            return CR.BEFORE
        elif ort is ORT.RIGHT:
            return CR.AFTER
        else:
            return CR.MATCH

    def _compare_edge_with_edge(self, edge1: HalfEdge, edge2: HalfEdge) -> CR:
        upper1, lower1 = edge1.upper_and_lower
        upper2, lower2 = edge2.upper_and_lower
        if upper1.point == upper2.point and lower1.point == lower2.point:
            return CR.MATCH
        elif lower2.y <= upper1.y <= upper2.y:
            return self._compare_vertex_with_edge(upper1, edge2)
        elif lower1.y <= upper2.y <= upper1.y:
            upper2_cr = self._compare_vertex_with_edge(upper2, edge1)
            if upper2_cr is CR.BEFORE:
                return CR.AFTER
            elif upper2_cr is CR.AFTER:
                return CR.BEFORE
            else:
                return CR.MATCH
        else:
            raise ValueError(f"The y-ranges of half-edges {edge1} and {edge2} don't intersect.")

Next, we define a [Python Enum](https://docs.python.org/3.9/library/enum.html) to distinguish between the different vertex types introduced in the lecture.
Intuitively, the types are named after what happens at vertex events during the downward plane sweep with respect to the intersection of the horizontal sweep line with the input polygon (see \[1, p. 50\]):
A vertex is called ...

* a *start vertex* if a new connected component of that intersection starts,

* an *end vertex* if one such component ends,

* a *split vertex* if one such component splits into two,

* a *merge vertex* if two such components merge into one,

* a *regular vertex* in all other cases.

For convenience, we write a class that computes the type of a vertex and stores it along with the vertex's incoming and outgoing polygon half-edges in anticlockwise direction.
Note that \[1\] refers to the incoming and outcoming edge of a vertex $v_i$ as $e_{i-1}$ and $e_i$, respectively (see \[1, p. 51\]).
The class additionally stores a half-edge that can be used for connecting the vertex to another via a diagonal.
It's initially set to the outgoing half-edge, but may be changed by the upcoming algorithm depending on context.

In [ ]:
class VertexType(Enum):
    START = auto()
    END = auto()
    SPLIT = auto()
    MERGE = auto()
    REGULAR = auto()

VT = VertexType

class VertexInfo:
    event_queue_comparator = EventQueueComparator()

    def __init__(self, vertex: Vertex):
        self.incoming_edge: HalfEdge = vertex.edge.prev
        self.outgoing_edge: HalfEdge = vertex.edge
        self.connection_edge: HalfEdge = vertex.edge

        prev_neighbour = self.incoming_edge.origin
        next_neighbour = self.outgoing_edge.destination
        prev_cr = self.event_queue_comparator.compare(prev_neighbour, vertex)
        next_cr = self.event_queue_comparator.compare(next_neighbour, vertex)
        if prev_cr is CR.AFTER and next_cr is CR.AFTER:
            if vertex.point.orientation(prev_neighbour.point, next_neighbour.point) is ORT.RIGHT:
                self.vertex_type = VT.START
            else:
                self.vertex_type = VT.SPLIT
        elif prev_cr is CR.BEFORE and next_cr is CR.BEFORE:
            if vertex.point.orientation(prev_neighbour.point, next_neighbour.point) is ORT.RIGHT:
                self.vertex_type = VT.END
            else:
                self.vertex_type = VT.MERGE
        else:
            self.vertex_type = VT.REGULAR

Let's finally dive into the actual plane sweep partitioning algorithm, whose goal is to create $y$-monotone subpolygons by adding diagonals that remove split and merge vertices.

First the algorithm initialises the status structure and the event queue using the BinaryTreeDict type.
That enables the event queue to store the discussed information for each of the promptly inserted polygon vertices.
The yet empty status structure will in turn store the information of every inserted half-edge's *helper*, i.e. the lowest vertex above the current sweep line position such that the horizontal line segment between the half-edge and the vertex is contained in the polygon (see \[1, p. 52\]).

After initialisation, the downward plane sweep handles all event vertices depending on their types using the approach from the lecture, which was adopted from \[1, pp. 53–54\].
There are two additional implementation details to take note of:

* Adding diagonals is done using half-edges, as explained in the previous section on Recursive Triangulation.
  Here, the connection half-edges come from the stored vertex information.
  These half-edges need to be changed in certain scenarios:
  Consider the case that an event vertex gets connected to a vertex above it via a diagonal.
  The added diagonal then partitions the polygon into two subpolygons, which both contain the event vertex.
  However, the connection half-edge from the event vertex information is incident to only one of the two subpolygon interiors, meaning the event vertex needs a different connection half-edge from the perspective of the other subpolygon.
  In order to support this, the *MonotonePartitioning._add_diagonal(...)* method returns not only the diagonal half-edge originating in the event vertex, but also a new version of the event vertex information with that half-edge as its connection half-edge.
  The handler methods for event vertices use the new version whenever appropriate.

* In preparation for the triangulation step in the next section, we again keep track of each subpolygon through a half-edge incident to its interior.
  To this end, notice that every subpolygon created by the downward plane sweep either contains the outgoing half-edge of an end vertex or gets closed off from containing any such half-edges.
  The latter happens if and only if an end vertex, a merge vertex or a regular vertex is connected to an above merge vertex via a diagonal.
  So taking suitable half-edges corresponding to those diagonals as well as the outgoing half-edges of all end vertices yields exactly one representative half-edge for each subpolygon.

In [ ]:
def monotone_partitioning(polygon: DoublyConnectedSimplePolygon) -> PointSequence:
    return MonotonePartitioning(polygon).partition()[0]

class MonotonePartitioning:
    def __init__(self, polygon: DoublyConnectedSimplePolygon):
        if polygon.has_diagonals():
            raise ValueError("Input polygon already has diagonals.")
        elif not polygon.is_simple():
            raise ValueError("Input polygon isn't simple.")
        self._polygon = polygon

        self._diagonal_points = PointSequence()
        self._representative_edges: list[HalfEdge] = []

        self._status_structure: BinaryTreeDict[HalfEdge, VertexInfo] = BinaryTreeDict(StatusStructureComparator())
        self._event_queue: BinaryTreeDict[Vertex, VertexInfo] = BinaryTreeDict(VertexInfo.event_queue_comparator)

        for vertex in self._polygon.vertices():
            self._event_queue.insert(vertex, VertexInfo(vertex))

    def partition(self) -> tuple[PointSequence, list[HalfEdge]]:
        while not self._event_queue.is_empty():
            event_vertex, event_vertex_info = self._event_queue.pop_first()
            handler_name = f"_handle_{str.lower(event_vertex_info.vertex_type.name)}_vertex"
            handler: Callable[[Vertex, VertexInfo], None] = getattr(self, handler_name)
            handler(event_vertex, event_vertex_info)
            if event_vertex_info.incoming_edge is event_vertex_info.outgoing_edge.prev:
                self._diagonal_points.animate(event_vertex.point)    # Animate point even if no diagonal was added.

        return self._diagonal_points, self._representative_edges

    def _handle_start_vertex(self, _: Vertex, event_vertex_info: VertexInfo):
        self._status_structure.insert(event_vertex_info.outgoing_edge, event_vertex_info)

    def _handle_end_vertex(self, _: Vertex, event_vertex_info: VertexInfo):
        _, inc_helper_info = self._status_structure.delete(event_vertex_info.incoming_edge)
        if inc_helper_info.vertex_type is VT.MERGE:
            diagonal, _ = self._add_diagonal(event_vertex_info, inc_helper_info)
            self._representative_edges.append(diagonal)
        self._representative_edges.append(event_vertex_info.outgoing_edge)

    def _handle_split_vertex(self, event_vertex: Vertex, event_vertex_info: VertexInfo):
        left_status_edge, left_helper_info = self._status_structure.search_predecessor(event_vertex)
        _, new_event_vertex_info = self._add_diagonal(event_vertex_info, left_helper_info)
        self._status_structure.update(left_status_edge, lambda _: new_event_vertex_info)
        self._status_structure.insert(event_vertex_info.outgoing_edge, event_vertex_info)

    def _handle_merge_vertex(self, event_vertex: Vertex, event_vertex_info: VertexInfo):
        _, inc_helper_info = self._status_structure.delete(event_vertex_info.incoming_edge)
        if inc_helper_info.vertex_type is VT.MERGE:
            diagonal, _ = self._add_diagonal(event_vertex_info, inc_helper_info)
            self._representative_edges.append(diagonal)
        left_status_edge, left_helper_info = self._status_structure.search_predecessor(event_vertex)
        if left_helper_info.vertex_type is VT.MERGE:
            diagonal, event_vertex_info = self._add_diagonal(event_vertex_info, left_helper_info)
            self._representative_edges.append(diagonal.twin)
        self._status_structure.update(left_status_edge, lambda _: event_vertex_info)

    def _handle_regular_vertex(self, event_vertex: Vertex, event_vertex_info: VertexInfo):
        is_interior_to_the_right, inc_helper_info = self._status_structure.delete(event_vertex_info.incoming_edge)
        if is_interior_to_the_right:
            if inc_helper_info.vertex_type is VT.MERGE:
                diagonal, _ = self._add_diagonal(event_vertex_info, inc_helper_info)
                self._representative_edges.append(diagonal)
            self._status_structure.insert(event_vertex_info.outgoing_edge, event_vertex_info)
        else:
            left_status_edge, left_helper_info = self._status_structure.search_predecessor(event_vertex)
            if left_helper_info.vertex_type is VT.MERGE:
                diagonal, event_vertex_info = self._add_diagonal(event_vertex_info, left_helper_info)
                self._representative_edges.append(diagonal.twin)
            self._status_structure.update(left_status_edge, lambda _: event_vertex_info)

    def _add_diagonal(self, event_vertex_info: VertexInfo, helper_info: VertexInfo) -> tuple[HalfEdge, VertexInfo]:
        diagonal = self._polygon.add_diagonal(event_vertex_info.connection_edge, helper_info.connection_edge)
        self._diagonal_points.append(diagonal.origin.point)
        self._diagonal_points.append(diagonal.destination.point)
        new_event_vertex_info = copy.copy(event_vertex_info)
        new_event_vertex_info.connection_edge = diagonal
        return diagonal, new_event_vertex_info

The asymptotic running time of Monotone Partitioning is in $O(n \log(n))$ and its data structures require space in $O(n)$.

In [ ]:
visualisation_tool.register_algorithm(
    "Mon. Partitioning", monotone_partitioning,
    MonotonePartitioningMode(animate_sweep_line = True, vertex_radius = 0)
)

Animating this algorithm on its own shows how diagonals are added as the sweep line passes the event vertices.

TODO: Add static image.

In [ ]:
visualisation_tool.display()

***Takeaways:***

* This time, we've encountered a plane sweep algorithm that knows all event vertices beforehand and doesn't require a dynamic status structure comparator.

* The DCEL is a flexible data structure, though one should be careful about possible changes in its internal structure when adding new edges.

### 3.3. Monotone Triangulation

We'll now tackle the missing piece, that is triangulating the $y$-monotone subpolygons.
For each subpolygon, the vertices on its left and right boundary chains are sorted from top to bottom thanks to the $y$-monotonicity.
The algorithm first merges both chains into a single descending vertex sequence.
A nice description for the next steps is provided by \[2, p. 32\]:
"The idea behind the triangulation algorithm is quite simple:
Try to triangulate *everything* you can \[above\] the current vertex by adding diagonals, and then remove the triangulated region from further consideration.
The trickiest aspect of implementing this idea is finding a clean invariant that characterizes the *untriangulated region* that lies \[above the current vertex\]."
Note that this quote has been slightly adapted, since \[2\] uses $x$-monotone subpolygons rather than $y$-monotone ones.

As seen in the lecture, a suitable invariant for the untriangulated region is that it has the shape of an upside-down funnel.
To be more specific, it consists of a single subpolygon edge on one side and a chain of vertices with interior angles above $180^\circ$ on the other side (see also \[1, p. 56\]).
This funnel is maintained through a *stack*.
Because we need to use appropriate connection half-edges for diagonals just like in the previous sections, our implementation stores half-edges on the stack instead of vertices.
For each half-edge, an orientation that signifies whether the half-edge is on the left or right boundary chain of the subpolygon is stored too.
This orientation is used to compare the current half-edge with the half-edge on top of the stack as well as to determine connection half-edges.
Here, the *MonotoneSupporter.add_diagonal(...)* method returns the correct half-edge for a potential connection with the next half-edge on top of the stack, which depends on the orientation of the half-edge previously popped from the stack.

In [ ]:
def monotone_triangulation(polygon: DoublyConnectedSimplePolygon) -> PointSequence:
    diagonal_points, representative_edges = MonotonePartitioning(polygon).partition()
    supporter = MonotoneSupporter(polygon, diagonal_points)

    for edge_sequence in supporter.get_edge_sequences(representative_edges):
        stack = edge_sequence[0:2]

        for current_edge, current_ort in edge_sequence[2:-1]:
            connection_edge = current_edge
            popped_edge, popped_ort = stack.pop()
            if current_ort is not popped_ort:
                cached_edge, cached_ort = popped_edge, popped_ort
                while stack:
                    connection_edge = supporter.add_diagonal(connection_edge, popped_edge, popped_ort)
                    popped_edge, popped_ort = stack.pop()
            else:
                current_point = current_edge.origin.point
                while stack:
                    ort = popped_edge.origin.point.orientation(current_point, stack[-1][0].origin.point)
                    if ort is not current_ort:
                        break
                    popped_edge, popped_ort = stack.pop()
                    connection_edge = supporter.add_diagonal(connection_edge, popped_edge, popped_ort)
                cached_edge, cached_ort = popped_edge, popped_ort
            if current_ort is ORT.LEFT:
                stack.append((current_edge.prev, cached_ort))
                stack.append((current_edge, current_ort))
            else:
                stack.append((cached_edge, cached_ort))
                stack.append((cached_edge.prev, current_ort))

        connection_edge = edge_sequence[-1][0]
        stack.pop()
        popped_edge, popped_ort = stack.pop()
        while stack:
            connection_edge = supporter.add_diagonal(connection_edge, popped_edge, popped_ort)
            popped_edge, popped_ort = stack.pop()

    return diagonal_points

class MonotoneSupporter:
    def __init__(self, polygon: DoublyConnectedSimplePolygon, diagonal_points: PointSequence):
        self._polygon = polygon
        self._diagonal_points = diagonal_points
        self._diagonal_points.reset_animations()    # Remove animations of points with no diagonal.

    def get_edge_sequences(self, representative_edges: list[HalfEdge]) -> Iterator[list[tuple[HalfEdge, ORT]]]:
        for representative_edge in representative_edges:
            topmost_edge = representative_edge
            edge = representative_edge.next
            while edge is not representative_edge:
                if VertexInfo.event_queue_comparator.compare(edge.origin, topmost_edge.origin) is CR.BEFORE:
                    topmost_edge = edge
                edge = edge.next

            edge_sequence: list[tuple[HalfEdge, ORT]] = [(topmost_edge, ORT.LEFT)]
            left_chain_edge = topmost_edge.next
            right_chain_edge = topmost_edge.prev
            while True:
                cr = VertexInfo.event_queue_comparator.compare(left_chain_edge.origin, right_chain_edge.origin)
                if cr is CR.BEFORE:
                    edge_sequence.append((left_chain_edge, ORT.LEFT))
                    left_chain_edge = left_chain_edge.next
                else:
                    edge_sequence.append((right_chain_edge, ORT.RIGHT))
                    if cr is CR.MATCH:
                        break
                    right_chain_edge = right_chain_edge.prev

            yield edge_sequence

    def add_diagonal(self, connection_edge: HalfEdge, popped_edge: HalfEdge, popped_ort: ORT) -> HalfEdge:
        diagonal = self._polygon.add_diagonal(connection_edge, popped_edge)
        self._diagonal_points.append(diagonal.origin.point)
        self._diagonal_points.append(diagonal.destination.point)

        if popped_ort is ORT.LEFT:
            return connection_edge
        return diagonal

The presented algorithm can triangulate a $y$-monotone polygon using $O(n)$ operations.
Its overall running time for general polygons is in $O(n \log(n))$, since it uses Monotone Partitioning as a preprocessing step to compute $y$-monotone subpolygons.

In [ ]:
visualisation_tool.register_algorithm(
    "Mon. Triangulation", monotone_triangulation,
    MonotonePartitioningMode(animate_sweep_line = False, vertex_radius = 0)
)

The animations of Monotone Triangulation show the diagonals added by Monotone Partitioning before triangulating the resulting subpolygons one after another.

TODO: Add static image.

In [ ]:
visualisation_tool.display()

***Takeaways:***

* Special cases have a greater potential for simple and efficient algorithms. Sometimes, this can be leveraged by reducing the general case to a special one.

* Even when designing a conceptually simple algorithm, translating an intuitive idea into a formal invariant can be somewhat tricky.

### 3.4. Gallery Guarding

We conclude this notebook by applying the presented polygon triangulation algorithms to the **art gallery problem**:
A gallery in the form of a simple polygon is to be guarded by security cameras.
Such a camera can be placed anywhere in the polygon and is able to rotate, guarding every point within the polygon such that the line segment between the camera and the point is contained in the polygon.
The goal is to find a small set of camera positions that together guard the whole polygon.
Finding a solution with as few cameras as possible is unfortunately NP-hard.
Hence, the lecture proposed finding a worst-case optimal solution with at most $\lfloor n/3 \rfloor$ cameras for polygons with $n$ vertices.

A solution of that kind is straightforward to compute using a triangulation of the input polygon:
First construct a **3-colouring** of the triangulated polygon, which assigns one of three colours to each polygon vertex such that all triangles have exactly one vertex of every colour.
Then place the cameras at those vertices whose colour appears the least.
Constructing a 3-colouring can be done with the help of the triangulation's *dual graph*, which is explained by \[1, pp. 47–48\].
The following implementation is based on a less illustrative but easier to implement approach from \[3\].
Essentially, it traverses the polygon vertices in anticlockwise order and assigns colours based on their degree parity.

In [ ]:
Triangulator = Callable[[DoublyConnectedSimplePolygon], PointSequence]

def recursive_guarding(polygon: DoublyConnectedSimplePolygon) -> PointSequence:
    return guarding(polygon, recursive_triangulation)

def monotone_guarding(polygon: DoublyConnectedSimplePolygon) -> PointSequence:
    return guarding(polygon, monotone_triangulation)

def guarding(polygon: DoublyConnectedSimplePolygon, triangulator: Triangulator) -> PointSequence:
    diagonal_points = triangulator(polygon)
    diagonal_points.clear()    # Clear diagonal points without removing animation information.

    topmost_vertex = polygon.topmost_vertex
    current_vertex = topmost_vertex.edge.destination
    coloured_vertices = ([topmost_vertex], [current_vertex], [])
    prev_colour, current_colour = 0, 1

    next_vertex = current_vertex.edge.destination
    while next_vertex is not topmost_vertex:
        has_current_vertex_odd_degree = True
        incident_edge = current_vertex.edge.prev.twin
        while incident_edge is not current_vertex.edge:
            has_current_vertex_odd_degree = not has_current_vertex_odd_degree
            incident_edge = incident_edge.prev.twin

        next_colour = prev_colour if has_current_vertex_odd_degree else 3 - prev_colour - current_colour
        coloured_vertices[next_colour].append(next_vertex)

        prev_colour, current_colour = current_colour, next_colour
        current_vertex, next_vertex = next_vertex, next_vertex.edge.destination

    camera_vertices = min(coloured_vertices, key = lambda vertices: len(vertices))
    camera_points = get_camera_points(camera_vertices, polygon.vertices_acw())

    return diagonal_points + camera_points

def get_camera_points(camera_vertices: list[Vertex], all_vertices: Iterator[Vertex]) -> PointSequence:
    camera_points = PointSequence()
    i = 0
    for vertex in all_vertices:
        if i < len(camera_vertices) and vertex.point == camera_vertices[i].point:
            camera_points.append(vertex.point)
            i += 1
        else:
            camera_points.animate(vertex.point)    # Animate point even if no camera is placed there.

    return camera_points

The 3-colouring algorithm visits all $n$ vertices and all edges, of which there are $n$ polygon edges and $n - 3$ diagonals, at most twice.
Thus, the overall worst-case running time complexity of (Gallery) Guarding is dominated by the used triangulation algorithm:
It's in $O(n^2)$ for Recursive Triangulation and in $O(n \log(n))$ for Monotone Triangulation.

In [ ]:
visualisation_tool.register_algorithm("Rec. Guarding", recursive_guarding, ArtGalleryMode())
visualisation_tool.register_algorithm("Mon. Guarding", monotone_guarding, ArtGalleryMode())

Having registered both variants for visualisation, you can use our tool to see how the underlying triangulation affects camera positions.
It's possible to find polygons in which Recursive Guarding places less cameras than Monotone Guarding and vice versa.
The *Prongs* example instance is the worst case presented in the lecture, where the 3-colouring approach optimally places one camera per prong (see also \[1, p. 48\]).
Then again, there exist polygons in which it places far more cameras than necessary.

TODO: Add static image.

In [ ]:
visualisation_tool.display()

## 4. References

\[1\] Mark de Berg, Otfried Cheong, Marc van Kreveld, and Mark Overmars. *Computational Geometry: Algorithms and Applications*, 3rd Edition, 2008.

\[2\] David M. Mount. [*CMSC 754: Computational Geometry*](https://www.cs.umd.edu/class/spring2020/cmsc754/Lects/cmsc754-spring2020-lects.pdf), 2020.

\[3\] Ali A. Kooshesh, and Bernard M. E. Moret. *Three-coloring the Vertices of a Triangulated Simple Polygon*, Pattern Recognition 25(4), p. 443, 1992.